<a href="https://colab.research.google.com/github/Aihikk/FakeNewsDetection/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [100]:
data_fake = pd.read_csv('Fake.csv', engine='python', on_bad_lines='skip')
data_true = pd.read_csv('True.csv')

In [101]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [102]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [103]:
data_fake["class"]=0
data_true["class"]=1

In [104]:
data_fake.shape , data_true.shape

((23481, 5), (21417, 5))

In [105]:
data_fake_manual_testing = data_fake.tail(20)
for i in range(23480, 23460, -1):
  data_fake.drop([i], axis=0, inplace=True)

data_true_manual_testing = data_true.tail(20)
for i in range(21416, 21406, -1):
  data_true.drop([i], axis=0, inplace=True)

In [106]:
data_fake_manual_testing.head()

,title,text,subject,date,class
23461,REPORT: ‘Federal Government Escalated the Viol...,KILLED: Rancher and protest spokesman Robert ...,Middle-east,"January 28, 2016",0
23462,"BOILER ROOM – Oregon Standoff, Cuddle Parties,...",Tune in to the Alternate Current Radio Network...,Middle-east,"January 28, 2016",0
23463,"Eyewitness Says Feds Ambushed Bundys, 100 Shot...",Patrick Henningsen 21st Century Wire UPDATE: 1...,Middle-east,"January 27, 2016",0
23464,Episode #119 – SUNDAY WIRE: ‘You Know the Dril...,Episode #119 of SUNDAY WIRE SHOW finally resum...,Middle-east,"January 24, 2016",0
23465,‘There’ll be boots on the ground’: US making n...,21st Century Wire says Various parties in Wash...,Middle-east,"January 23, 2016",0


In [107]:
data_true_manual_testing.head()

,title,text,subject,date,class
21397,Germany's Schulz says he would demand U.S. wit...,BERLIN (Reuters) - The leader of Germany s Soc...,worldnews,"August 23, 2017",1
21398,Blunt instrument? What a list of banned articl...,SHANGHAI (Reuters) - An old review of an acade...,worldnews,"August 23, 2017",1
21399,Saudi police release teenager detained for dan...,DUBAI (Reuters) - A 14-year-old boy who was de...,worldnews,"August 22, 2017",1
21400,"The People's Princess, Britons work to keep me...",LONDON (Reuters) - Abdul Daoud spilt most of t...,worldnews,"August 23, 2017",1
21401,"Argentina labor unions protest job losses, Mac...",BUENOS AIRES (Reuters) - Argentina s main labo...,worldnews,"August 22, 2017",1


In [108]:
data_merge = pd.concat([data_fake , data_true], axis=0)
data_merge.head(20)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [109]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [110]:
data = data_merge.drop(['title', 'subject', 'date'], axis=1)

In [111]:
data.isnull().sum()

,0
text,0
class,0


In [112]:
data = data.sample(frac = 1)

In [113]:
data.head()

,text,class
20556,(This Aug 31 story corrects location of Burkh...,1
17212,Jesse Watters confronts the board of superviso...,0
19287,"MUNICH/POTSDAM, Germany (Reuters) - More than ...",1
16529,SARAJEVO (Reuters) - Thousands of pensioners f...,1
14854,Oh the irony of a terror attack by Muslims tak...,0


In [114]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [115]:
data.columns

Index(['text', 'class'], dtype='object')

In [116]:
data.head()

,text,class
0,(This Aug 31 story corrects location of Burkh...,1
1,Jesse Watters confronts the board of superviso...,0
2,"MUNICH/POTSDAM, Germany (Reuters) - More than ...",1
3,SARAJEVO (Reuters) - Thousands of pensioners f...,1
4,Oh the irony of a terror attack by Muslims tak...,0


In [117]:
def wordopt(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return text

In [118]:
data['text'] = data['text'].apply(wordopt)

In [119]:
x = data["text"]
y = data["class"]

In [120]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [121]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [122]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [123]:
pred_lr = LR.predict(xv_test)

In [124]:
LR.score(xv_test, y_test)

0.9849335829544441

In [125]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5889
           1       0.98      0.98      0.98      5328

    accuracy                           0.98     11217
   macro avg       0.98      0.98      0.98     11217
weighted avg       0.98      0.98      0.98     11217



In [126]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [127]:
pred_dt = DT.predict(xv_test)

In [128]:
DT.score(xv_test, y_test)

0.9966122849246679

In [129]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5889
           1       1.00      0.99      1.00      5328

    accuracy                           1.00     11217
   macro avg       1.00      1.00      1.00     11217
weighted avg       1.00      1.00      1.00     11217



In [130]:
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [131]:
predict_gb = GBC.predict(xv_test)

In [132]:
GBC.score(xv_test, y_test)

0.9946509761968441

In [133]:
print(classification_report(y_test, predict_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5889
           1       0.99      1.00      0.99      5328

    accuracy                           0.99     11217
   macro avg       0.99      0.99      0.99     11217
weighted avg       0.99      0.99      0.99     11217



In [134]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [135]:
pred_rf = RFC.predict(xv_test)

In [136]:
RFC.score(xv_test, y_test)

0.990906659534635

In [137]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5889
           1       0.99      0.99      0.99      5328

    accuracy                           0.99     11217
   macro avg       0.99      0.99      0.99     11217
weighted avg       0.99      0.99      0.99     11217



In [138]:
def output_lable(n):
  if n == 0:
    return "Fake News"
  elif n == 1:
    return "True News"

def manual_testing(news):
  testing_news = {"text":[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(wordopt)
  new_x_test = new_def_test["text"]
  new_xv_test = vectorization.transform(new_x_test)
  pred_LR = LR.predict(new_xv_test)
  pred_DT = DT.predict(new_xv_test)
  pred_GBC = GBC.predict(new_xv_test)
  pred_RFC = RFC.predict(new_xv_test)

  print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                              output_lable(pred_DT[0]),
                                                                                                              output_lable(pred_GBC[0]),
                                                                                                              output_lable(pred_RFC[0])))

In [148]:
news =  str(input())
manual_testing(news)

As U.S. budget fight looms, Republicans flip their fiscal script


LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News
